<a href="https://colab.research.google.com/github/hawc2/Text-Analysis-with-Python/blob/master/LDA_Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling

In [ ]:
# source - https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

# Installing Packages

In [ ]:
!pip install pyLDAvis

In [ ]:
from __future__ import unicode_literals, print_function
import re
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
from importlib import reload
from pprint import pprint
#import sys
#get_ipython().run_line_magic('matplotlib', 'inline')


# Load Files

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
# Or manually upload
#or add function from Github or extracted features

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Test.csv to Test.csv


OSError: ignored

In [ ]:
df = pd.read_csv("/content/Test.csv", usecols=['Username', 'content'])

In [ ]:
print(df)

                                                content     Username
0     I totally misheard and thought this was going ...       Matt D
1     Great movie that shares a very wide range of e...      Marks V
2     Despite a minimal narrative arc, the film does...      Jared D
3     Kathryn Bigelow's The Hurt Locker is something...      Brett C
4                           Worth the best picture win.  Christian H
...                                                 ...          ...
1007  Intense look into the lives of an American ord...  Westleigh Q
1008  if you've heard anything about this movie you'...      Tyson P
1009  O.V.R 8....can't believe I've waited this long...       Bryn D
1010  Explosive, dominating, an emotional firestorm,...      Aaron J
1011  The Hurt Locker is easily the best "dramatizat...       Joel D

[1012 rows x 2 columns]


In [ ]:
data = df.content.values.tolist()

In [ ]:
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['I totally misheard and thought this was going to be a sequel about Bernie '
 'Focker. Still good, though!']


<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \s
<input>:2: Dep

# Clean Texts

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
      yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(data_words)



[['totally', 'misheard', 'and', 'thought', 'this', 'was', 'going', 'to', 'be', 'sequel', 'about', 'bernie', 'focker', 'still', 'good', 'though'], ['great', 'movie', 'that', 'shares', 'very', 'wide', 'range', 'of', 'emotions'], ['despite', 'minimal', 'narrative', 'arc', 'the', 'film', 'does', 'contain', 'some', 'gripping', 'action', 'scenes', 'and', 'sense', 'of', 'that', 'keeps', 'us', 'guessing', 'until', 'the', 'end'], ['kathryn', 'bigelows', 'the', 'hurt', 'locker', 'is', 'something', 'felt', 'was', 'different', 'in', 'war', 'film', 'rather', 'than', 'focusing', 'the', 'film', 'in', 'the', 'reasons', 'why', 'the', 'country', 'goes', 'into', 'war', 'but', 'rather', 'the', 'reason', 'why', 'soldier', 'wants', 'to', 'be', 'in', 'the', 'war', 'it', 'details', 'on', 'the', 'experience', 'of', 'these', 'soldiers', 'in', 'their', 'placement', 'in', 'iraq', 'this', 'is', 'also', 'the', 'film', 'that', 'made', 'kathryn', 'bigelow', 'and', 'oscar', 'contending', 'director', 'and', 'has', 'pro

# Building Dictionary and Corpus

In [ ]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)], [(16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1)], [(1, 1), (19, 1), (22, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 2), (41, 1), (42, 1)], [(0, 1), (1, 30), (2, 6), (6, 1), (10, 12), (11, 1), (13, 15), (15, 2), (16, 1), (17, 5), (19, 21), (22, 16), (23, 3), (25, 3), (29, 1), (31, 19), (34, 1), (35, 1), (37, 1), (38, 2), (40, 50), (41, 1), (43, 1), (44, 1), (45, 3), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 5), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 5), (64, 1), (65, 5), (66, 1), (67, 1), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 3), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 1), (83, 2), (84, 2), (85, 1), (

# Create Topic Model

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=2,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)



/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

# Create Visualization

In [ ]:
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(vis, '/content/LDAviz.html')